### Perform removing important content and perform sentiment analysis

In [55]:
import re
import ollama
import pandas as pd
import json
import re
from typing import List

In [56]:
## Read and set the csv of scraped data

df = pd.read_csv("info/carwow_scraped_data_full.csv")
df.head()

,url,title,price,rating,tag,review
0,https://www.carwow.co.uk/mg/mg-4,MG4 EV REVIEW & PRICES,"€37,142",8/10,new,Is the MG4 EV a good car? Is the MG4 EV a good...
1,https://www.carwow.co.uk/mg/5,MG5 EV REVIEW & PRICES,"€37,738",6/10,new,Is the MG5 EV a good car? Is the MG5 EV a good...
2,https://www.carwow.co.uk/mg/zs-ev,MG ZS EV Review & Prices,"€38,616",7/10,new,Is the MG ZS EV a good car? Is the MG ZS EV a ...
3,https://www.carwow.co.uk/mg/mg3,MG3 REVIEW & PRICES,"€22,224",9/10,new,Is the MG3 a good car? Is the MG3 a good car? ...
4,https://www.carwow.co.uk/mg/gs,MG GS Review and Prices,"€21,561",5/10,new,The MG GS is a medium-sized family SUV that’s ...


In [57]:
## Example of line
df.iloc[0]

url                        https://www.carwow.co.uk/mg/mg-4
title                                MG4 EV REVIEW & PRICES
price                                               €37,142
rating                                                 8/10
tag                                                     new
review    Is the MG4 EV a good car? Is the MG4 EV a good...
Name: 0, dtype: object

In [58]:
## Creation of a list of reviews to help in the iteration later

reviews = {}
for idx, row in df.iterrows():
    # Remove 'Review & Prices' or similar suffixes from the title to get the model name
    title = row['title']
    model = title.replace('Review & Prices', '').replace('REVIEW & PRICES', '').replace('Review and Prices', '').strip()
    reviews[idx] = {
        'url': row['url'],
        'brand': row['url'].split('/')[3].upper(),
        'model': model,
        'price': str(row['price']).replace(',', '.').replace('€', ''),
        'rating': row['rating'],
        'condition': row['tag'],
        'review': row['review']
    }

In [59]:
reviews[6]['url'].split('/')

['https:', '', 'www.carwow.co.uk', 'mg', 'zs', '2018']

In [60]:
def clean_review(text):
    if pd.isnull(text):
        return text

    # Rimuovi domande introduttive generiche
    text = re.sub(r'(Is the [A-Za-z0-9\s\-]+ (a|an) good car\?\s*)+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(Should I buy the [A-Za-z0-9\s\-]+\?\s*)+', '', text, flags=re.IGNORECASE)

    # Rimuovi call to action e promozionali
    text = re.sub(r'(Check out our .*?deals.*?\.|You can also check out .*?\.|You can even sell your current car through .*?\.|Want to sell your car online.*?\.|Find out how much you can save.*?\.|You can browse used .*?\.|Remember that .*? can even help you sell your car online too\.)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'(Sound interesting\? Check here for our best .*? deals.*?\.|You could also check out other used .*? here.*?\.|And remember that .*? can even help you sell your car online too\.)', '', text, flags=re.IGNORECASE)

    # Rimuovi FAQ e titoli ripetuti (ma NON i paragrafi che seguono)
    text = re.sub(r'([A-Za-z0-9\s\-]+ FAQs|FAQs|How much is the [A-Za-z0-9\s\-]+\?)\s*', '', text, flags=re.IGNORECASE)

    # Rimuovi specifiche tecniche pure (solo se sono su una riga separata)
    text = re.sub(r'^(Range|Efficiency|Battery size|Boot|Power outputs|Charge time|CO2 emissions|MPG, emissions and tax|0-60mph time|Top speed|Charging|Boot space|Running costs|Company car|Tax|Insurance|Warranty|Recalls):.*?(\.|\n)', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # Rimuovi riferimenti a premi, survey, rating, ecc.
    text = re.sub(r'(Carwow Car of the Year Awards.*?\.|Driver Power reliability survey.*?\.|Euro NCAP safety testing.*?\.|Euro NCAP safety tests.*?\.|Euro NCAP safety rating.*?\.|Euro NCAP.*?\.|Benefit-in-Kind tax.*?\.|Vehicle Excise Duty.*?\.|Warranty.*?\.|recall.*?\.|survey.*?\.|reviewed separately.*?\.|FAQs.*?\.|FAQs)', '', text, flags=re.IGNORECASE)

    # Rimuovi prezzi e valute isolate
    text = re.sub(r'£[0-9,]+', '', text)
    text = re.sub(r'€[0-9,]+', '', text)
    text = re.sub(r'\$[0-9,]+', '', text)

    # Rimuovi link, email, numeri di telefono
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\+?\d[\d\-\s]{7,}\d', '', text)

    # Rimuovi frasi tipo "Read more", "See also", "For more information"
    text = re.sub(r'(Read more.*?\.|See also.*?\.|For more information.*?\.|Find out more.*?\.)', '', text, flags=re.IGNORECASE)

    # Rimuovi disclaimer e note legali
    text = re.sub(r'(All information correct at time of publication.*?\.|Terms and conditions apply.*?\.|Subject to status.*?\.|Representative example.*?\.|Your mileage may vary.*?\.)', '', text, flags=re.IGNORECASE)

    # Rimuovi frasi di confronto generiche
    text = re.sub(r'(compared to the [A-Za-z0-9\s\-]+|versus the [A-Za-z0-9\s\-]+|alternative to the [A-Za-z0-9\s\-]+)', '', text, flags=re.IGNORECASE)

    # Rimuovi parentesi quadre e il loro contenuto
    text = re.sub(r'\[.*?\]', '', text)

    # Rimuovi righe che iniziano con titoli/sezioni non utili
    text = re.sub(r'^(Verdict|Summary|Should I buy.*|What’s it like.*|Pros and cons|Key features|Running costs|Company car|Tax|Insurance|Warranty|Recalls|Boot space|Charging|Range|Final thoughts|Our verdict|In summary|To sum up|Pros:|Cons:|Conclusion|Overview|Introduction|At a glance|Highlights|Quick facts|Performance and drive comfort|Space and practicality|Interior style, infotainment and accessories|Safety and reliability|Alternatives to the .*)\s*$', '', text, flags=re.IGNORECASE | re.MULTILINE)

    # Rimuovi righe molto brevi (meno di 3 parole)
    text = '\n'.join([line for line in text.split('\n') if len(line.strip().split()) > 2])

    # Rimuovi caratteri speciali ripetuti (es: ----, ====, ###)
    text = re.sub(r'[-=_#]{3,}', '', text)

    # Rimuovi tutti i simboli particolari tranne . , : ;
    text = re.sub(r"[^\w\s\.\,\:\;]", "", text)

    # Rimuovi simboli e spazi ripetuti
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()

    return text

In [61]:
for i, review in reviews.items():
    print("Length before: ", len(review['review']))
    review['review'] = clean_review(review['review'])
    print("Length after: ", len(review['review']))

Length before:  15127
Length after:  13841
Length before:  11707
Length after:  11292
Length before:  11305
Length after:  10742
Length before:  12742
Length after:  11491
Length before:  3004
Length after:  2897
Length before:  13334
Length after:  12824
Length before:  14177
Length after:  13571
Length before:  14333
Length after:  13669
Length before:  12554
Length after:  11746
Length before:  12158
Length after:  11756
Length before:  13914
Length after:  13029
Length before:  13247
Length after:  12579
Length before:  12471
Length after:  11956
Length before:  12765
Length after:  12175
Length before:  15364
Length after:  14719
Length before:  13782
Length after:  13094
Length before:  13658
Length after:  13059
Length before:  13878
Length after:  13034
Length before:  2043
Length after:  1840
Length before:  4518
Length after:  4267
Length before:  15408
Length after:  14768
Length before:  14354
Length after:  13583
Length before:  7595
Length after:  7336
Length before:  152

In [62]:
reviews[2]

{'url': 'https://www.carwow.co.uk/mg/zs-ev',
 'brand': 'MG',
 'model': 'MG ZS EV',
 'price': '38.616',
 'rating': '7/10',
 'condition': 'new',
 'review': 'The MG ZS EV is an electric SUV that goes up against the likes of the Hyundai Kona Electric and Kia Niro EV both of which are now wellestablished options at the affordable end of the electric car market. Its a bit like going for a pair of cheap Bluetooth headphones. It might not be from the obvious choice of brands, but it promises to do a similar job at a lower cost. Since its revival as a massproduction manufacturer, MGs models havent been the most exciting to look at. The new ZS EV is in a similar vein, but this updated version now comes with a blankedout grille and LED headlights as part of a tweaked front end. There is also a new rear bumper with altered lights. MG has also improved the interior design, with the main addition being a new 10.1inch infotainment touchscreen that has updated software to make it much smoother than be

In [63]:
## Selection of the model to use in the sentiment analysis

model_llm = {
    0: "llama3.1:8b",
    1: "qwen3:4b",
    2: "llama3.2:3b-instruct-q8_0",
    3: "deepseek-r1:7b",
    4: "mistral:7b-instruct",
    5: "deepseek-r1:8b",
    6: "llama3.1:8b-instruct-q5_K_M",
    7: "gemma3:4b",
    8: "gemma3:4b-it-q8_0",
    9: "llama3.1:8b-instruct-q2_K",
    10: "llama3.2:3b-instruct-q5_K_M",
    11: "gemma3:1b",
    12: "llama3.1:8b-instruct-fp16",
    13: "llama3.2:1b-instruct-q8_0",
    14: "llama3.2:1b-instruct-fp16"
}
model_used = model_llm[11]
print(model_used)

gemma3:1b


In [64]:
## Read and set the system information

with open("info/system_information.txt", "r") as f:
    system_information = f.read()

In [65]:
## Read and set the system information

with open("info/summarize.txt", "r") as f:
    system_information_summary = f.read()

In [66]:
def extract_insights_ollama_summarize(review_dict):
    prompt_text = json.dumps(review_dict, ensure_ascii=False)
    
    response = ollama.chat(
        model=model_used,
        messages=[
            {"role": "system", "content": system_information_summary},
            {"role": "user", "content": prompt_text}
        ],
    )
    
    return response['message']['content']

In [83]:
def extract_insights_ollama(review_dict, model_to_use):
    prompt_text = json.dumps(review_dict, ensure_ascii=False)
    
    response = ollama.chat(
        model=model_to_use,
        messages=[
            {"role": "system", "content": system_information},
            {"role": "user", "content": prompt_text}
        ],
    )
    
    return response['message']['content']


In [68]:
## Function to extract and clean the response

def clean_response(response):
    response = re.sub(
        r"'Price':\s*[$£]?([0-9]{1,3}(?:,[0-9]{3})*(?:\.\d+)?|\d+\.?\d*)",
        lambda m: f"'Price': {m.group(1).replace(',', '')}",
        response
    )
    response = re.sub(r"'Overall':\s*([0-9]+(?:\.\d+)?)\s*/\s*10", r"'Overall': \1", response)
    response = re.sub(r',\s*}', '}', response) 
    return response
    
def extract_backtick_block(s):
    # First try to match triple backticks
    m = re.search(r'```(.*?)```', s, flags=re.DOTALL)
    # If not found, try single backticks
    if not m:
        m = re.search(r'`(.*?)`', s, flags=re.DOTALL)
    return m.group(1).strip() if m else s.strip()

In [69]:
import unicodedata

def remove_unicode(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

In [70]:
print(remove_unicode(reviews[3]['review']))

The MG3 used to be a bit of a joke the old one was the lowestranked car on Carwow but the new model aims to change all that. Now, instead of being antiquated, its right up there with the best small cars for performance, efficiency and equipment. Like an air fryer from the middle of Lidl, the MG3s feature set is as good as any of its alternatives and its not immediately clear where the moneys been saved to bring it in at such a low price. Now that its a full selfcharging hybrid car , the MG3 no longer targets the very cheapest on the market like the Dacia Sandero or Citroen C3 Origin . Instead, its more like a cutprice , but either dont want or cant have a fully electric model. The MG3 wont win awards for its practicality its not cramped, but its no better than average in this regard with a boot thats about halfway between the Yaris and the Clio in capacity. Rear seat space is moderate, too, though annoyingly the rear seats dont split when they fold which does limit overall versatility.

In [ ]:
results_summarize = []

# Calculate the total number of items in reviews
total_items = len(reviews)
# Divide into 4 batches
batch_size = total_items // 4 + (1 if total_items % 4 > 0 else 0)

print(f"Total items: {total_items}, processing in 4 batches of approximately {batch_size} items each")

for batch_num in range(4):
    start_idx = batch_num * batch_size
    end_idx = min((batch_num + 1) * batch_size, total_items)
    
    print(f"\nProcessing batch {batch_num + 1}/4 (items {start_idx} to {end_idx-1})")
    
    batch_results = []
    for i in range(start_idx, end_idx):
        review = reviews[i]
        print(f"Processing item {i+1}/{total_items} ({(i+1)/total_items*100:.1f}%)...")
        review['review'] = remove_unicode(review['review'])
        
        max_retries = 5
        attempts = 0
        proc_response = ""
        
        while attempts < max_retries:
            try:
                raw = extract_insights_ollama_summarize(review)
                response = extract_backtick_block(raw)
                proc_response = clean_response(response)
                
                if proc_response == "" or len(proc_response) < 600:
                    print(f"Processed response too short: {len(proc_response)} characters")
                    attempts += 1
                    print(f"Attempt {attempts}/{max_retries} failed, retrying...")
                    if attempts >= max_retries:
                        print(f"Failed after {max_retries} attempts, using last response")
                else:
                    print(f"Successfully processed ({len(proc_response)} characters)")
                    break
            except Exception as e:
                print(f"Error during processing: {str(e)}")
                attempts += 1
                if attempts >= max_retries:
                    print(f"Failed after {max_retries} attempts due to errors")
                    proc_response = f"Error processing review: {str(e)}"
                print(f"Retrying... ({attempts}/{max_retries})")
        
        batch_results.append(proc_response)
    
    # Add batch results to the main results list
    results_summarize.extend(batch_results)
    
    # Save intermediate results after each batch
    print(f"Batch {batch_num + 1} complete. Saved {len(batch_results)} summaries.")
    with open(f"results_summarize_batch_{batch_num+1}.txt", "w", encoding="utf-8") as f:
        for item in batch_results:
            f.write(str(item) + "\n\n---\n\n")

print(f"All processing complete. Total summaries: {len(results_summarize)}")

In [ ]:
with open("info/results_summarize.txt", "w", encoding="utf-8") as f:
    for item in results_summarize:
        f.write(str(item) + "\n\n")

In [71]:
def review_to_single_line(review):
    return re.sub(r'\s+', ' ', review).strip()

In [74]:
import glob

file_list = sorted(glob.glob("results_summarize_batch_*.txt"))

result = []
delimiter = "---"

for filename in file_list:
    with open(filename, "r", encoding='utf-8') as f:
        temp = f.read()
        blocks = temp.split(delimiter)
        for line in blocks:
            line = line.replace('"', '').replace('“', '').replace("Here's a summary of the car review:", '').replace("Here's a summarized version of the car review:", '')
            single_line = review_to_single_line(line)
            result.append(single_line)    

In [75]:
result[:7]

["The MG4 EV presents a compelling alternative to established rivals, offering a stylish and practical driving experience. While it doesn't boast the same raw power as some competitors, it excels in comfort and refinement. The interior is well-designed and surprisingly spacious for a compact EV, though the infotainment system can be frustrating to use. Handling is smooth and responsive, making it a pleasant car to drive. Its compact size also contributes to ease of maneuverability. While the range is respectable, it’s not exceptional, and charging times can take a bit longer than some rivals. Overall, the MG4 EV offers a good value proposition, combining stylish design, comfortable ride, and acceptable performance.”",
 'The MG5 EV offers a comfortable and surprisingly capable electric estate, despite some minor design updates. While the interior lacks the premium feel of some rivals, the ride is comfortable and the electric range is respectable, reaching 250 miles with the SE version. 

In [77]:
for i, review in reviews.items():
    review['review'] = result[i]

In [79]:
reviews

{0: {'url': 'https://www.carwow.co.uk/mg/mg-4',
  'brand': 'MG',
  'model': 'MG4 EV',
  'price': '37.142',
  'rating': '8/10',
  'condition': 'new',
  'review': "The MG4 EV presents a compelling alternative to established rivals, offering a stylish and practical driving experience. While it doesn't boast the same raw power as some competitors, it excels in comfort and refinement. The interior is well-designed and surprisingly spacious for a compact EV, though the infotainment system can be frustrating to use. Handling is smooth and responsive, making it a pleasant car to drive. Its compact size also contributes to ease of maneuverability. While the range is respectable, it’s not exceptional, and charging times can take a bit longer than some rivals. Overall, the MG4 EV offers a good value proposition, combining stylish design, comfortable ride, and acceptable performance.”"},
 1: {'url': 'https://www.carwow.co.uk/mg/5',
  'brand': 'MG',
  'model': 'MG5 EV',
  'price': '37.738',
  'rati

In [84]:
## Selection of the model to use in the sentiment analysis

model_llm = {
    0: "llama3.1:8b",
    1: "qwen3:4b",
    2: "llama3.2:3b-instruct-q8_0",
    3: "deepseek-r1:7b",
    4: "mistral:7b-instruct",
    5: "deepseek-r1:8b",
    6: "llama3.1:8b-instruct-q5_K_M",
    7: "gemma3:4b",
    8: "gemma3:4b-it-q8_0",
    9: "llama3.1:8b-instruct-q2_K",
    10: "llama3.2:3b-instruct-q5_K_M",
    11: "gemma3:1b",
    12: "llama3.1:8b-instruct-fp16",
    13: "llama3.2:1b-instruct-q8_0",
    14: "llama3.2:1b-instruct-fp16",
    15: "mistral:7b-instruct-q2_K"
}
model_used = model_llm[15]

In [85]:
import ast

results = []

for i, review in reviews.items():
    print(f"Processing {i+1}/{len(reviews)}…")
    review['review'] = remove_unicode(review['review'])
    raw = extract_insights_ollama(review, model_used)
    response = extract_backtick_block(raw)
    proc_response = clean_response(response)

    print(f"Process response: {proc_response}")
    result_dict = {}
    try:
        result_dict = ast.literal_eval(proc_response)
    except Exception as e:
        print(f"Result: {result_dict}")
        print(f"Parsing error: {e}")
        result_dict = {}

    retries = 0
    while len(result_dict) != 17 and retries < 3:
        print(f"Invalid format ({len(result_dict)} keys). Retrying…")
        retry_system_message = (
            system_information +
            "\nYou did NOT return the dictionary in the exact format. ONLY output the dictionary below, enclosed in triple backticks. "
            "DO NOT write any explanation, summary, or bullet points. ONLY the dictionary. "
            "If you do not follow the output format, your answer will be discarded."
        )
        # print("SYSTEM PROMPT:", retry_system_message)
        # print("USER PROMPT:", json.dumps(review, ensure_ascii=False))
        raw = ollama.chat(
            model=model_used,
            messages=[
                {"role": "system", "content": retry_system_message},
                {"role": "user", "content": json.dumps(review, ensure_ascii=False)}
            ],
        )['message']['content']
        response = extract_backtick_block(raw)
        proc_response = clean_response(response)

        print(f"Process response: {proc_response}")

        try:
            result_dict = ast.literal_eval(proc_response)
        except Exception as e:
            print(f"Result: {result_dict}")
            print(f"Retry parsing error: {e}")
            result_dict = {}
        retries += 1

    results.append(result_dict)

Processing 1/206…
Process response: {
    "Brand": "MG",
    "Model": "MG4 EV",
    "Drive": "Positive", 
    "Interior quality": "Positive", 
    "Infotainment system": "Negative", 
    "Comfort": "Positive", 
    "Performance": "Medium", 
    "Handling": "Positive", 
    "Practicality": "Not mentioned", 
    "Reliability": "Not mentioned", 
    "Safety": "Not mentioned", 
    "Quality of construction": "Not mentioned", 
    "Noise": "Not mentioned", 
    "Engine": "Not mentioned", 
    "Price": "€37.142", 
    "Condition": "new",
    "Overall": "8"
}
Processing 2/206…
Process response: {
    "Brand": "MG",
    "Model": "ZS EV",
    "Drive": "Medium", 
    "Interior quality": "Positive", 
    "Infotainment system": "Positive", 
    "Comfort": "Negative", 
    "Performance": "Not mentioned", 
    "Handling": "Not mentioned", 
    "Practicality": "Medium", 
    "Reliability": "Not mentioned", 
    "Safety": "Medium", 
    "Quality of construction": "Medium", 
    "Noise": "Negative", 
 

In [86]:
## Just for example

test_dict = {
    'Brand': 'MG',
    'Model': 'ZS',
    'Drive': 'Medium', 
    'Quality of interior': 'Positive', 
    'Infotainment system': 'Positive', 
    'Comfort': 'Negative', 
    'Performance': 'Medium', 
    'Handling': 'Not mentioned', 
    'Practicality': 'Positive', 
    'Reliability': 'Not mentioned', 
    'Safety': 'Negative', 
    'Quality of construction': 'Medium', 
    'Noise': 'Negative', 
    'Engine': 'Medium', 
    'Price': 18.605,
    'tag': 'new',
    'Overall': 6
}

len(test_dict)

17

In [87]:
## We use the test_dict keys to create the columns of the Dataframe, then we create it.
df_new = pd.DataFrame(columns = test_dict.keys())

for result_dict in results:
    df_new.loc[len(df_new)] = result_dict
    
df_new

,Brand,Model,Drive,Quality of interior,Infotainment system,Comfort,Performance,Handling,Practicality,Reliability,Safety,Quality of construction,Noise,Engine,Price,tag,Overall
0,MG,MG4 EV,Positive,NaN,Negative,Positive,Medium,Positive,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,€37.142,NaN,8
1,MG,ZS EV,Medium,NaN,Positive,Negative,Not mentioned,Not mentioned,Medium,Not mentioned,Medium,Medium,Negative,Not mentioned,€18.605,NaN,5
2,MG,ZS,Medium,NaN,Positive,Negative,Medium,Not mentioned,Positive,Not mentioned,Negative,Medium,Positive,Positive,€38.616,NaN,7
3,MG,ZS EV,Positive,NaN,Medium,Negative,Positive,Not mentioned,Negative,Not mentioned,Medium,Not mentioned,Positive,Medium,€18.605,NaN,6
4,MG,GS,Medium,NaN,Negative,Not mentioned,Not mentioned,Medium,Positive,Not mentioned,Negative,Negative,Not mentioned,Diesel,€21.561,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,BYD,Seal,Negative,NaN,Not mentioned,Medium,Positive,Positive,Positive,Not mentioned,Not mentioned,Negative,Not mentioned,Not mentioned,€35.106,NaN,4
202,BYD,Seal U,Negative,NaN,Positive,Negative,Medium,Not mentioned,Not mentioned,Positive,Positive,Positive,Not mentioned,Not mentioned,38.189,NaN,6
203,BYD,Dolphin,Positive,NaN,Medium,Positive,Positive,Not mentioned,Positive,Positive,Medium,Positive,Not mentioned,Not mentioned,€42.705,NaN,8
204,LEAPMOTOR,T03,Medium,NaN,Not mentioned,Negative,Negative,Not mentioned,Positive,Not mentioned,Medium,Medium,Negative,Medium,18.714,NaN,5


In [88]:
## Saving the Dataframe as a csv.

df_new.to_csv("info/car_review_new_test.csv", index = False)